In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [2]:
movie_ratings = pd.read_csv('Movie Rating.csv')

In [3]:
movie_names = pd.read_csv('movies_metadata.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
movie_users = pd.read_csv('ratings.csv')

In [5]:
movie_names = movie_names[['original_title', 'id']]

In [6]:
movie_names.columns

Index(['original_title', 'id'], dtype='object')

In [7]:
movie_users.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [8]:
movie_names = movie_names[~movie_names['id'].str.contains("(\d{4}-\d\d-\d\d)")]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [9]:
movie_names['id'] = movie_names['id'].astype(int)

In [10]:
movie_ppl = movie_users.merge(movie_names, left_on = 'movieId', right_on='id').drop(columns=['id','timestamp'])

In [11]:
imdb = pd.read_csv('data.csv')

In [12]:
imdb.head()

,Title,Imdb Score,Metascore
0,Pulp Fiction,8.9,94.0
1,The Amazing Spider-Man 2,6.6,53.0
2,The Shawshank Redemption,9.3,80.0
3,Star Wars: Episode IV - A New Hope,8.6,90.0
4,Back to the Future,8.5,87.0


In [13]:
movies = imdb.merge(movie_ppl, left_on='Title', right_on='original_title')

In [14]:
movies.drop(columns = ['movieId', 'original_title'], inplace=True)

In [15]:
movies.head()

,Title,Imdb Score,Metascore,userId,rating
0,Pulp Fiction,8.9,94.0,229,4.0
1,Pulp Fiction,8.9,94.0,314,3.5
2,Pulp Fiction,8.9,94.0,375,4.5
3,Pulp Fiction,8.9,94.0,481,5.0
4,Pulp Fiction,8.9,94.0,609,5.0


In [16]:
len(movies['Title'].unique())

2066

In [17]:
len(movies['userId'].unique())

255488

In [18]:
len(movies)

6331233

In [19]:
movies.dropna(inplace=True)

In [20]:
grouped = movies.groupby(['userId', 'Title']).mean()

In [21]:
grouped.head()

Imdb Score  Metascore  rating
userId Title                                                   
1      Fools Rush In                    6.00       37.0     4.0
       License to Wed                   5.30       25.0     4.0
       Rocky Balboa                     7.10       63.0     5.0
       Sleepless in Seattle             6.80       72.0     5.0
2      A Nightmare on Elm Street        6.35       55.5     4.0

In [22]:
test_df = grouped.loc[1]

In [23]:
test_df

,Imdb Score,Metascore,rating
Title,,,
Fools Rush In,6.0,37.0,4.0
License to Wed,5.3,25.0,4.0
Rocky Balboa,7.1,63.0,5.0
Sleepless in Seattle,6.8,72.0,5.0


In [24]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
squared_errors = []

In [26]:
equations = {}

In [27]:
for i in movies['userId'].unique():
    if(len(grouped.loc[i]) > 1):
        train, test = train_test_split(grouped.loc[i])
        reg = linear_model.LinearRegression().fit(train[['Imdb Score', 'Metascore']], train['rating'])
        error = (test['rating'][0] - reg.predict(test[['Imdb Score', 'Metascore']])[0])**2
        squared_errors.append(error)
        equations[i] = reg.coef_

In [28]:
print('Mean Squared Error ' + str(np.mean(squared_errors)))

Mean Squared Error 5.063970078361853e+24


In [29]:
imdb[imdb['Title'] == 'The Dark Knight']

,Title,Imdb Score,Metascore
35,The Dark Knight,9.0,84.0


In [30]:
squared_errors

[0.020490816883981675,
 0.20936720378906362,
 2.453566838500094,
 3.4362442277415783,
 0.029624939649964203,
 0.2761180319784762,
 2.1191706398656165,
 0.6153804852763443,
 0.821759757593126,
 1.9909132718808409,
 3.8627660537630906,
 2.0779401549574907,
 2.6110757671134963,
 0.03387656120304304,
 0.05828832577797342,
 0.05701874502496638,
 0.8780648955932541,
 2.957679799872833,
 0.10941553505979656,
 0.33563527568605384,
 0.3995400649559995,
 1.1442494694584537,
 0.42029304447395777,
 0.68373462792841,
 0.012489498272433919,
 1.2003037454537502,
 0.9981245334809179,
 2.2328140590033594,
 0.29331995371585634,
 0.011930506169508241,
 1.789192584470046,
 0.058161483180829615,
 3.408391379096915,
 2.4164100962460773,
 0.04524304036786257,
 0.03798203690527974,
 1.36891503655399,
 0.05251129068789306,
 0.07355624422461515,
 2.5252286486119733,
 2.149420993404515,
 0.010426495488478321,
 1.9558242939139385,
 0.308723660591342,
 0.013678975753989355,
 0.06406955341858084,
 1.810580582460309

In [31]:
options = ["Pulp Fiction", "Star Wars: Episode IV - A New Hope", "The Shawshank Redemption", "Toy Story", "The Amazing Spider-Man 2", "Mean Girls", "The Silence of the Lambs", "Jurassic Park", "The Lord of the Rings: The Fellowship of the Ring", "The Hangover"]

In [32]:
tester = imdb[imdb['Title'].isin(options)]
tester

,Title,Imdb Score,Metascore
0,Pulp Fiction,8.9,94.0
1,The Amazing Spider-Man 2,6.6,53.0
2,The Shawshank Redemption,9.3,80.0
3,Star Wars: Episode IV - A New Hope,8.6,90.0
7,The Silence of the Lambs,8.6,85.0
8,Jurassic Park,8.1,68.0
11,Toy Story,8.3,95.0
20,The Lord of the Rings: The Fellowship of the Ring,8.8,92.0
26,Mean Girls,7.0,66.0
40,The Hangover,7.7,73.0


In [33]:
current_user = np.array

In [34]:
intercept = None

In [35]:
def initialize(ratings):
    df = tester.copy()
    df['Ratings'] = ratings
    reg = linear_model.LinearRegression().fit(df[['Imdb Score', 'Metascore']], df['Ratings'])
    global current_user
    current_user = reg.coef_
    global intercept
    intercept = reg.intercept_

In [36]:
initialize([4,2,5,3,5,3,4,2,4,5])

In [37]:
def personalized_rating(title):
    if intercept:
        print("Your Personalized Rating for " + title + " is " + str(float(imdb[imdb['Title'] == title]['Imdb Score']*current_user[0] + imdb[imdb['Title'] == title]['Imdb Score']*current_user[1]) + intercept))
    else:
        print("Your Personalized Rating for " + title + " is " + str(float(imdb[imdb['Title'] == title]['Imdb Score']*current_user[0] + imdb[imdb['Title'] == title]['Imdb Score']*current_user[1])))


In [38]:
personalized_rating("The Dark Knight")

Your Personalized Rating for The Dark Knight is 4.959301672786298


In [39]:
personalized_rating("Inception")

Your Personalized Rating for Inception is 4.850531462250029


In [40]:
personalized_rating('Saw')

Your Personalized Rating for Saw is 4.1979101990324175


In [41]:
personalized_rating("Batman Begins")

Your Personalized Rating for Batman Begins is 4.524220830641223


In [43]:
initialize([3,3,3,3,3,3,3,3,3,3])

In [44]:
personalized_rating("The Dark Knight")

Your Personalized Rating for The Dark Knight is 3.0


In [45]:
personalized_rating('Star Trek')

Your Personalized Rating for Star Trek is 3.0


In [46]:
initialize([5,3,5,5,0,4,4,3,5,5])

In [47]:
personalized_rating("Paul Blart: Mall Cop")

Your Personalized Rating for Paul Blart: Mall Cop is 4.00141532849016


In [48]:
personalized_rating("The Room")

Your Personalized Rating for The Room is 3.9980432123327603


In [49]:
initialize([4,5,5,4,5,4,5,5,2,5])

In [50]:
personalized_rating("Mean Girls 2")

Your Personalized Rating for Mean Girls 2 is 3.0560282455048324


In [51]:
initialize([2,1,5,3,5,5,4,3,5,5])

In [52]:
personalized_rating("Shrek")

Your Personalized Rating for Shrek is 6.478088754983689


In [53]:
personalized_rating("The Martian")

Your Personalized Rating for The Martian is 6.645049009565533
